# Compton Imager

## Import Tools

In [75]:
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from itertools import product, combinations
from numpy import arctan2, sqrt
import numexpr as ne
import matplotlib as mpl
import matplotlib.colors as colors
from matplotlib.mlab import bivariate_normal
from matplotlib.colorbar import ColorbarBase, make_axes_gridspec
from matplotlib import cm
import scipy
import math 
from scipy.special import erf
from scipy.special import erfinv
from scipy import integrate
import cmath
import numpy as np
import heapq

## Defining The Functions In Use

In [76]:
def quartic(phi_in,x0_in,y0_in,z0_in,Theta_C):
    
    #Constants needed in the calculation
    c_open = np.tan(Theta_C)
    rho = (x0_in*x0_in + y0_in*y0_in + z0_in*z0_in - r*r)
    eta = (1 + (1/(c_open*c_open)))
    
    #original coefficients to the quartic equation. This is used to test in the quartic equation calculator
    a0 = eta*eta
    b0 = -4*eta*(x0_in*np.cos(phi_in) + y0_in*np.sin(phi_in))
    c0 = (2*x0_in*np.cos(phi_in) + 2*y0_in*np.sin(phi_in))*(2*x0_in*np.cos(phi_in) + 2*y0_in*np.sin(phi_in)) - ((4*z0_in*z0_in)/(c_open*c_open)) + 2*eta*rho
    d0 = -4*rho*(x0_in*np.cos(phi_in) + y0_in*np.sin(phi_in))
    e0 = rho*rho
    
    coefficients_of_quartic = [a0,b0,c0,d0,e0]
    final_roots = np.roots(coefficients_of_quartic)
    
    #print('these are also the final roots',final_roots)
    #print ('The coefficients of the quartic equation are', 'a0 =',a0,'b0 =',b0,'c0 =',c0,'d0 =',d0,'e0 =',e0)
    return final_roots

def Rotation_x(Angle_1):
        Rotation_Angle_x = Angle_1
        R_X = [[1,0,0],[0,np.cos(Rotation_Angle_x),-np.sin(Rotation_Angle_x)],[0,np.sin(Rotation_Angle_x),np.cos(Rotation_Angle_x)]]
        return R_X

def Rotation_y(Angle_2):
        Rotation_Angle_y = Angle_2
        R_Y = [[np.cos(Rotation_Angle_y),0,np.sin(Rotation_Angle_y)],[0,1,0],[-np.sin(Rotation_Angle_y),0,np.cos(Rotation_Angle_y)]]
        return R_Y

def Rotation_z(Angle_3):
        Rotation_Angle_z = Angle_3
        R_Y = [[np.cos(Rotation_Angle_z),-np.sin(Rotation_Angle_z),0],[np.sin(Rotation_Angle_z),np.cos(Rotation_Angle_z),0],[0,0,1]]
        return R_Y

def Rotation_Euclid(Latitude,Longitude,Twist):
    Latitude = theta_rot
    Longitude = phi_rot
    Twist = 0
    R_Euclid_1st_Row = [np.cos(phi_rot)*np.cos(theta_rot)*np.cos(Twist) - np.sin(phi_rot)*np.sin(Twist),-np.sin(Twist)*np.cos(theta_rot)*np.cos(phi_rot) - np.sin(phi_rot)*np.cos(Twist), np.sin(theta_rot)*np.cos(phi_rot)]
    R_Euclid_2nd_Row = [np.sin(phi_rot)*np.cos(theta_rot)*np.cos(Twist) + np.cos(phi_rot)*np.sin(Twist),-np.sin(Twist)*np.cos(theta_rot)*np.sin(phi_rot) + np.cos(phi_rot)*np.cos(Twist), np.sin(theta_rot)*np.sin(phi_rot)]
    R_Euclid_3rd_Row = [-np.sin(theta_rot)*np.cos(Twist), -np.sin(theta_rot)*np.sin(Twist), np.cos(theta_rot)]
    R_Euclid = [R_Euclid_1st_Row,R_Euclid_2nd_Row,R_Euclid_3rd_Row]
    return R_Euclid

def Rotation(theta_zf,phi_zf):
        theta_r = theta_zf   
        phi_r = phi_zf  + np.pi/2 
        #print (theta_r, phi_r)
        k_x = np.cos(phi_r) #weights for rotation in x and y
        k_y = np.sin(phi_r)
        k_z = 0
        R = [[np.cos(theta_r)+(k_x**2)*(1-np.cos(theta_r)),(k_x*k_y)*(1-np.cos(theta_r))-(k_z)*(np.sin(theta_r)),(k_x*k_z)*(1-np.cos(theta_r))+(k_y)*(np.sin(theta_r))], [(k_y*k_x)*(1-np.cos(theta_r)) + (k_z)*(np.sin(theta_r)), np.cos(theta_r) + (k_y**2)*(1-np.cos(theta_r)) , (k_y*k_z)*(1-np.cos(theta_r)) - (k_x)*(np.sin(theta_r))] , [(k_x*k_z)*(1-np.cos(theta_r))- (k_y)*(np.sin(theta_r)),(k_z*k_y)*(1-np.cos(theta_r)) + (k_x)*(np.sin(theta_r)), np.cos(theta_r) + (k_z**2)*(1-np.cos(theta_r))]]               
        return R

def phi_value(x0,x2,y0,y2):
    #thinking out loud: These next series might have to go before the special cases. Incase they wipe out the special case values. 
        
    if (x0-x2) >= 0 and (y0-y2) >= 0:
        phi_out = np.arccos((x0 - x2)/(np.sqrt((x0 - x2)*(x0 - x2) + (y0-y2)*(y0-y2)))) 
   
    if (x0-x2) <= 0 and (y0-y2) >= 0:
        phi_out = np.arccos((x0 - x2)/(np.sqrt((x0 - x2)*(x0 - x2) + (y0-y2)*(y0-y2)))) 
    
    if (x0-x2) <= 0 and (y0-y2) <= 0:
        phi_out = np.arccos((x0 - x2)/(np.sqrt((x0 - x2)*(x0 - x2) + (y0-y2)*(y0-y2)))) + np.pi/2
    
    if (x0-x2) >= 0 and (y0-y2) <= 0:
        phi_out = np.arccos((x0 - x2)/(np.sqrt((x0 - x2)*(x0 - x2) + (y0-y2)*(y0-y2)))) + 3*np.pi/2
            
    
    
    #Special Cases
    if np.abs((y0-y2)) == 0 and y2 == 0 and (x0-x2)>= 0:
        phi_out = 0 
    
    if np.abs((y0-y2)) == 0 and y2 == 0 and (x0-x2) < 0:
        phi_out = np.pi 
    
    if np.abs((x0-x2)) <= 0.0001 and x2 == 0 and (y0-y2) > 0:
        phi_out = np.pi/2
    
    if np.abs((x0-x2)) <= 0.0001 and x2 == 0 and (y0-y2) < 0:
        phi_out = 3*np.pi/2
    
    if np.abs((x0-x2)) <= 0.0001 and x2 == 0 and np.abs((y0-y2)) == 0:
        phi_out = 0
       
    return phi_out

## These are the locations of the interactions and the angles they create

In [77]:
#These are all the initial inputs required for the algorithm to run. 
Theta_C = 30*(np.pi/180)
c_open = np.tan(Theta_C)

#These determine the rotation angle (They are the positions of the events inside the TPC)
x0 = 2#works from -8 to +8
y0 = 2 #works from -2 to +8. Reducing x0 to 0 helps this greatly. Plug in numbers to quartic to solve. 
z0 = 2*np.sqrt(2) #When this is straight down or up the phi value will be 90. THere are also some graphing errors that occur.


#why does changing x2,y2 and z2 change the position of the ring? What do they physically tell us about the system?
x2 = 1
y2 = 1
z2 = 1*np.sqrt(2)

if x0 == 0 and x2 == 0:
    x0 = 0.0001

#Finding the Rotation Angle required so that the initial vertex point is parallel to the z-axis
g = [x0-x2, y0-y2, z0-z2]
mag_g = np.sqrt (g[0]*g[0] + g[1]*g[1] + g[2]*g[2])
g_hat = g/mag_g
theta_vertex_rot = np.arccos(g[2]/mag_g)

#Rotation Angle based on points. 
theta_rot = np.arccos((z0-z2)/np.sqrt((x0-x2)**2 +(y0-y2)**2 + (z0-z2)**2)) 

phi_rot = phi_value(x0,x2,y0,y2)   

print ( 'this is the theta rotation angle', (theta_rot*180/np.pi), ' and this is the phi rotation', (phi_rot*180/np.pi))

this is the theta rotation angle 45.0  and this is the phi rotation 45.0


## Constants To Determine Error

In [78]:
#These corrispond to near field events occuring within the TPC. How far off the detectors center did they occur. 
r = 10

#Constants to determine the error spread function.
Delta_Theta = 3
Number_Of_Phi_values = 40
Step_Size = 1
total_points_plotted = Number_Of_Phi_values * Step_Size*(2*Delta_Theta+1) 
Energy_Value = 1.7 # Assign value of energy 
mu = Theta_C
delta_Y = y2-y0
delta_X = x2-x0
delta_Z = z2-z0
Depth_Uncertainty = 1
x_uncertainty = 2
y_uncertainty = 2
p = np.sqrt(6*(x_uncertainty**2 + y_uncertainty**2)) #seems to get error when p is small. 
sigma_theta_squared = (p**2)/(6*(delta_X**2 + delta_Y**2))
sigma_phi_squared = (1/(delta_X**2 + delta_Y**2 + delta_Z**2)**2)*((delta_Z**2)*((p**2)/6) + 2*(Depth_Uncertainty)*((delta_X)**2 + (delta_Y)**2))
sigma_phi_prime_squared = 2*(np.arcsin(np.sin(np.sqrt(sigma_phi_squared)/2)*np.cos(theta_rot)))

## Creating A Single Back Projected Cone

In [79]:
#The first part exists simply to produce a spread function that can help determine our error.     
vertex_initial = [x0,y0,z0]
vertex_shift_phi = np.dot(Rotation_z(-phi_rot),vertex_initial)
vertex_shift_theta = np.dot (Rotation_y(-theta_rot),vertex_shift_phi)

x0_rot = vertex_shift_theta[0]
y0_rot = vertex_shift_theta[1]
z0_rot = vertex_shift_theta[2]


#Initialize the arrays
theta_ring =[]
phi_ring = []

#Creating a Ring in Phi-Space from 0 to 2pi
phi_ring_1 = np.linspace(0, (2*np.pi), (Number_Of_Phi_values))

#Phi Ring is now 200 rows and 9 columns 
for i in range (0, Number_Of_Phi_values):
    phi_ring.append([phi_ring_1[i]]*(Step_Size*(2*Delta_Theta+1)))


#Determining the points of the Back projected Cone. Finding the Poisiton after they have been rotated. 
#Must also account for the shift in x0,y0,z0. You cannot simply add them over here. They need to be run through the quartic. 
#this is only the intial ring. No need to create a spread. 
p_set = []
x_set = []
y_set = []
z_set = []
final_phi_set = []

for i in range (len(phi_ring)):
    final_phi_set.append(phi_ring_1[i])
    final_phi_set.append(phi_ring_1[i])
    final_phi_set.append(phi_ring_1[i])
    final_phi_set.append(phi_ring_1[i])   

#Should the quartic inputs be negative?
for i in range (0,Number_Of_Phi_values):
    p_set.append(quartic(phi_ring_1[i],-x0_rot,-y0_rot,-z0_rot,Theta_C)) 

final_p_set = []
for i in range (len(p_set)):
    for j in range (len(p_set[0])):
        final_p_set.append(p_set[i][j])


        
#The way in which x,y,z is determined. For every single phi there are 4 different values. (symmetric)
y_phi_test = []

for i in range (len(final_p_set)):
    x_set.append(final_p_set[i]*np.cos(final_phi_set[i]))
    y_set.append(final_p_set[i]*np.sin(final_phi_set[i]))
    

for i in range (len(final_p_set)): 
    z_set.append(np.sqrt((x_set[i]*x_set[i] + y_set[i]*y_set[i])/(c_open*c_open)))

x_set[:] = [x + (x0_rot) for x in x_set]  
y_set[:] = [x + (y0_rot) for x in y_set]
z_set[:] = [x + (z0_rot) for x in z_set]


BPC_Cartesian = [x_set,y_set,z_set]

x_set_BPC, y_set_BPC, z_set_BPC = [],[],[]

for i in range (0,4):
    x_set_BPC.append(x_set[i::4])
    y_set_BPC.append(y_set[i::4])
    z_set_BPC.append(z_set[i::4])

    
    
Surface_Check = []

Total_On_Sphere = [0,0,0,0] #create a 4 x 1 empty matrix

Position_Rank = []
Total_Rank = []    
    
surface_allowed_error = .1
    
    
for i in range (0,4):
    for j in range (0,Number_Of_Phi_values):
        Surface_Check.append(np.sqrt(x_set_BPC[i][j]*x_set_BPC[i][j] + y_set_BPC[i][j]*y_set_BPC[i][j] + z_set_BPC[i][j]*z_set_BPC[i][j]))


Surface_Check_Reshaped = np.reshape(Surface_Check, (4,Number_Of_Phi_values))



for i in range (0,4):
    for j in range (0,Number_Of_Phi_values):
        if np.abs((r - (Surface_Check_Reshaped[i][j]))) < surface_allowed_error:
            Total_On_Sphere[i] = 1 + Total_On_Sphere[i] 
    Position_Rank.append(100*(Total_On_Sphere[i]/Number_Of_Phi_values)) #The smaller the better. 
   




Correct_Indicies = np.argpartition(Position_Rank, -2)[-2:]

if np.sum(Position_Rank)/100 > 2:
    Correct_Indicies = [1,3]

x_set_BPC_Final = np.concatenate((x_set_BPC[Correct_Indicies[0]],x_set_BPC[Correct_Indicies[1]]))
y_set_BPC_Final = np.concatenate((y_set_BPC[Correct_Indicies[0]],y_set_BPC[Correct_Indicies[1]]))
z_set_BPC_Final = np.concatenate((z_set_BPC[Correct_Indicies[0]],z_set_BPC[Correct_Indicies[1]]))

BPC_Final = [x_set_BPC_Final,y_set_BPC_Final,z_set_BPC_Final]

BPC_Rotated = np.dot(Rotation_Euclid(theta_rot,phi_rot,0),BPC_Final)  

x_BPC_Final = BPC_Rotated[0]
y_BPC_Final = BPC_Rotated[1]
z_BPC_Final = BPC_Rotated[2]

print (len(x_BPC_Final))

80


## Error Spread Function 

In [80]:
#determining the cartesian location of the axis of the cone 
#test for finding the axis of the cone. First create the shift and then rotate. 

x_cone_axis = np.mean(x_BPC_Final)
y_cone_axis = np.mean(y_BPC_Final)
z_cone_axis = np.mean(z_BPC_Final)

#Three points on the Plane
Source_C = [0,0,r] #first point (point C)
Point_A = [(x_cone_axis),(y_cone_axis),(z_cone_axis)] #second point (Point A)
Origin = [0,0,0] #third point (Point O)

#two vectors in plane OAC
vector1 = [Point_A[0] - Source_C[0], Point_A[1] - Source_C[1], Point_A[2] - Source_C[2]] 
vector2 = [Origin[0] - Source_C[0], Origin[1] - Source_C[1], Origin[2] - Source_C[2]]

#Finding the Normal Vector 
cross_product = [vector1[1] * vector2[2] - vector1[2] * vector2[1], -1 * (vector1[0] * vector2[2] - vector1[2] * vector2[0]), vector1[0] * vector2[1] - vector1[1] * vector2[0]]

#For the equation of a plane in 3D
a = cross_product[0]
b = cross_product[1]
c = cross_product[2]
d = - (cross_product[0] * Source_C[0] + cross_product[1] * Source_C[1] + cross_product[2] * Source_C[2])   

#Need to determine vector AB
x_values_of_AB = x_cone_axis - x_BPC_Final #Here.Subtracting fixed my issue. I am finding a vector between two points. Not figuring out a reference frame. 
y_values_of_AB = y_cone_axis - y_BPC_Final
z_values_of_AB = z_cone_axis - z_BPC_Final

Beta = np.arcsin(np.absolute(a*x_values_of_AB + b*y_values_of_AB + c*z_values_of_AB) / ((np.sqrt(a**2 + b**2 + c**2))*(np.sqrt(x_values_of_AB**2 + y_values_of_AB**2 + z_values_of_AB**2))))

#Can think of this as an amplitude used for width spread. np.sqrt(sigma_theta_squared) then np.sqrt(sigma_phi_squared)* there is one spread for every phi
Spread = np.sqrt(sigma_theta_squared)*np.cos(Beta)**2 + np.sqrt(sigma_phi_squared)*np.sin(Beta)**2

print ('this is the sigma theta weight', np.sqrt(sigma_theta_squared))
print ('this is the sigma phi weight', np.sqrt(sigma_phi_squared))

#print ((Spread))
#behavior of beta and spread seams to be normal. It is a sinusoidal function whose values rise and fall as such. Matched it with the original case with no shift. 

this is the sigma theta weight 2.0
this is the sigma phi weight 1.11803398875


## Applying the Error Spread and Creating Final Points

In [81]:
#Now we generalize to plot the entire thing. 
Spacing = []

#this loop provides us with an array that contains the error for the compton angle cone opening. It is an inverse error function.
for i in range (0,Step_Size*(2*Delta_Theta+1)):
    Spacing.append(erfinv(((((2*i+ 1)/(Step_Size*(2*Delta_Theta+1)) - 1)))))


#Create values for the error in compton angle 
for j in range (0, Number_Of_Phi_values):
    for i in range (0, Step_Size*(2*Delta_Theta+1)):
        theta_ring.append((Theta_C + ((Spacing[i]*Spread[j])*(np.pi/180)))) #Check to see if spacing*spread is in radians. 
        
print (len(theta_ring))   
c_open_f = []
for i in range (len(theta_ring)):
    c_open_f.append(np.tan(theta_ring[i]))


#Spread remains unaffected by the delta theta C values. Each phi value gets spread in theta space according to the spread times the inverse error function (gaussian). 
#These are individual inputs for theta. If the shift were to occur with these then there would need to be an equal number of quartic loops.
    
#Determining the points of the Back projected Cone. Finding the Poisiton after they have been rotated. 
#Must also account for the shift in x0,y0,z0. You cannot simply add them over here. They need to be run through the quartic. 
#this is only the intial ring. No need to create a spread. 
p_set_f = []
x_set_f = []
y_set_f = []
z_set_f = []
  

#This needs to have Recreation phi_ring_1 but repeating (2*Delta_Theta+1) times and the theta_C ring of equal dimensions fed to it. 
phi_ring_t =[]
final_phi_ring_t = []

for i in range (len(phi_ring)):
    for j in range (len(phi_ring[0])):
        phi_ring_t.append(phi_ring[i][j])

final_phi_set_f = []
for i in range (len(phi_ring_t)):
    final_phi_set_f.append(phi_ring_t[i])
    final_phi_set_f.append(phi_ring_t[i])
    final_phi_set_f.append(phi_ring_t[i])
    final_phi_set_f.append(phi_ring_t[i]) 

c_open_f_1 =[]
for i in range (len(c_open_f)):
    c_open_f_1.append(c_open_f[i])
    c_open_f_1.append(c_open_f[i])
    c_open_f_1.append(c_open_f[i])
    c_open_f_1.append(c_open_f[i])
    
for i in range (len(phi_ring_t)):
    p_set_f.append(quartic(phi_ring_t[i],-x0_rot,-y0_rot,-z0_rot,theta_ring[i])) 

final_p_set_f = []
for i in range (len(p_set_f)):
    for j in range (len(p_set_f[0])):
        final_p_set_f.append(p_set_f[i][j])

for i in range (len(final_p_set_f)):
    x_set_f.append(final_p_set_f[i]*np.cos(final_phi_set_f[i]))
    y_set_f.append(final_p_set_f[i]*np.sin(final_phi_set_f[i]))

#create an array for c_open_f
for i in range (len(final_p_set_f)): 
    z_set_f.append(np.sqrt((x_set_f[i]*x_set_f[i] + y_set_f[i]*y_set_f[i])/(c_open_f_1[i]*c_open_f_1[i])))
    
final_x_set_f = []
final_y_set_f = []
final_z_set_f = []

for i in range (0,(2*Delta_Theta+1)*Number_Of_Phi_values):
    final_x_set_f.append(x_set_f[Correct_Indicies[0] + 4*i])
    final_x_set_f.append(x_set_f[Correct_Indicies[1] + 4*i])
    #final_x_set_f.append(x_set_f[2 + 4*i])
    final_y_set_f.append(y_set_f[Correct_Indicies[0] + 4*i])
    final_y_set_f.append(y_set_f[Correct_Indicies[1] + 4*i])
    #final_y_set_f.append(y_set_f[2 + 4*i])
    final_z_set_f.append(z_set_f[Correct_Indicies[0] + 4*i])
    final_z_set_f.append(z_set_f[Correct_Indicies[1] + 4*i])
    #final_z_set_f.append(z_set_f[2 + 4*i])
    #print (np.sqrt(x_set_f[4*i]*x_set_f[4*i] + y_set_f[4*i]*y_set_f[4*i] + z_set_f[4*i]*z_set_f[4*i]))

final_x_set_f[:] = [x + x0_rot for x in final_x_set_f]
final_y_set_f[:] = [x + y0_rot for x in final_y_set_f]
final_z_set_f[:] = [x + z0_rot for x in final_z_set_f]

#these should be the points of the ring before a rotation occurs. 
BPC_Cartesian_f = [final_x_set_f,final_y_set_f,final_z_set_f]

#here comes the rotation
v_f2_f = np.dot(Rotation_Euclid(theta_rot,phi_rot,0),BPC_Cartesian_f)
x_f_f = v_f2_f[0]
y_f_f = v_f2_f[1]
z_f_f = v_f2_f[2]

280


## Plotting

In [82]:
# index_set = np.linspace (0,99,100)
# print (len(theta_ring))
# plt.plot(index_set,theta_ring, marker = 's') #should be a smooth gradient, but its not 


# plt.show()

In [83]:
#setting for plotting
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_aspect("equal")

#plot wireframe circle
u, v = np.mgrid[0:2*np.pi:40j, 0:np.pi:20j] #these determine the number of lines in the grid
x_grid=np.cos(u)*np.sin(v)
y_grid=np.sin(u)*np.sin(v) 
z_grid=np.cos(v) 
x_grid_1 = r*x_grid #not sure if I should shift these values with x0,y0,z0. Seems to be warping the circle
y_grid_1 = r*y_grid 
z_grid_1 = r*z_grid 
ax.plot_wireframe(x_grid_1, y_grid_1, z_grid_1, color="grey", linewidth = 0.25)

#Set Axis
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

#Origin
ax.scatter(x0,y0,z0, c ='blue', marker ='p')
ax.scatter(x2,y2,z2, c = 'pink', marker = 's')
ax.scatter(0,0,0, c = 'grey', marker = 'o')


#ax.scatter (x_BPC_Final[0:10], y_BPC_Final[0:10], z_BPC_Final[0:10],c ='blue', s=10,  marker = ',', lw = 0 )
ax.scatter (x_cone_axis, y_cone_axis, z_cone_axis,c ='red', s=10,  marker = ',', lw = 0 )



#ax.scatter (x_set, y_set, z_set,c ='blue', s=10,  marker = ',', lw = 0 )

#ax.scatter (r*np.sin(theta_rot)*np.cos(phi_rot), r*np.sin(theta_rot)*np.sin(phi_rot), r*np.cos(theta_rot),c ='red', s=40,  marker = ',', lw = 0 )
#ax.scatter(x0_rot, y0_rot, z0_rot, c = 'green',s = 30, marker = '*', lw= 0)


ax.scatter (x_f_f, y_f_f, z_f_f, c ='blue', s=10,  marker = ',', lw = 0 ) #something weird is happening with the spacing, figure out the spread. 




plt.show()



## Various Test Cells 

In [84]:
# #The first part exists simply to produce a spread function that can help determine our error.     
# vertex_initial = [x0,y0,z0]
# vertex_shift_phi = np.dot(Rotation_z(-phi_rot),vertex_initial)
# vertex_shift_theta = np.dot (Rotation_y(-theta_rot),vertex_shift_phi)

# x0_rot = vertex_shift_theta[0]
# y0_rot = vertex_shift_theta[1]
# z0_rot = vertex_shift_theta[2]


# #Initialize the arrays
# theta_ring =[]
# phi_ring = []

# #Creating a Ring in Phi-Space from 0 to 2pi
# phi_ring_1 = np.linspace(0, (2*np.pi), (Number_Of_Phi_values))

# #Phi Ring is now 200 rows and 9 columns 
# for i in range (0, Number_Of_Phi_values):
#     phi_ring.append([phi_ring_1[i]]*(Step_Size*(2*Delta_Theta+1)))


# #Determining the points of the Back projected Cone. Finding the Poisiton after they have been rotated. 
# #Must also account for the shift in x0,y0,z0. You cannot simply add them over here. They need to be run through the quartic. 
# #this is only the intial ring. No need to create a spread. 
# p_set = []
# x_set = []
# y_set = []
# z_set = []
# final_phi_set = []

# for i in range (len(phi_ring)):
#     final_phi_set.append(phi_ring_1[i])
#     final_phi_set.append(phi_ring_1[i])
#     final_phi_set.append(phi_ring_1[i])
#     final_phi_set.append(phi_ring_1[i])   

# #Should the quartic inputs be negative?
# for i in range (0,Number_Of_Phi_values):
#     p_set.append(quartic(phi_ring_1[i],-x0_rot,-y0_rot,-z0_rot,Theta_C)) 

# final_p_set = []
# for i in range (len(p_set)):
#     for j in range (len(p_set[0])):
#         final_p_set.append(p_set[i][j])


        
# #The way in which x,y,z is determined. For every single phi there are 4 different values. (symmetric)
# y_phi_test = []

# for i in range (len(final_p_set)):
#     x_set.append(final_p_set[i]*np.cos(final_phi_set[i]))
#     y_set.append(final_p_set[i]*np.sin(final_phi_set[i]))#Both the cos and sin of the final phi set works. (meaning they go negative to positive)
    
    

# for i in range (len(final_p_set)): 
#     z_set.append(np.sqrt((x_set[i]*x_set[i] + y_set[i]*y_set[i])/(c_open*c_open)))

# x_set[:] = [x + (x0_rot) for x in x_set]  
# y_set[:] = [x + (y0_rot) for x in y_set]
# z_set[:] = [x + (z0_rot) for x in z_set]

# #print (len(x_set), len(y_set), len(z_set))

# BPC_Cartesian = [x_set,y_set,z_set]

# #These are wrong. The x has only negative or so and so forth. 
# # x_test = BPC_Cartesian[0][::4]
# # y_test = BPC_Cartesian[1][::4]
# # z_test = BPC_Cartesian[2][::4]


# #here comes the rotation
# BPC_Rotated = np.dot(Rotation_Euclid(theta_rot,phi_rot,0),BPC_Cartesian)  #error. 


# # x_test_1 = BPC_Rotated[0][::4]
# # y_test_1 = BPC_Rotated[1][::4]
# # z_test_1 = BPC_Rotated[2][::4]


# # #Check to see the individual values of the BPC. 
# # faditest = BPC_Rotated[0][:50]
# # danytest = []
# # for i in range(len(faditest)):
# #     danytest.append("%.2f" % faditest[i])




# #final product of points B around the cone axis
# x_BPC = BPC_Rotated[0]
# y_BPC = BPC_Rotated[1]
# z_BPC = BPC_Rotated[2]

# x_BPC_Reshaped, y_BPC_Reshaped, z_BPC_Reshaped = [],[],[]

# for i in range (0,4):
#     x_BPC_Reshaped.append(x_BPC[i::4])
#     y_BPC_Reshaped.append(y_BPC[i::4])
#     z_BPC_Reshaped.append(z_BPC[i::4])

# # #not sure this is correct. 
# # x_BPC_Reshaped = np.reshape(x_BPC, (4,Number_Of_Phi_values))
# # y_BPC_Reshaped = np.reshape(y_BPC, (4,Number_Of_Phi_values))
# # z_BPC_Reshaped = np.reshape(z_BPC, (4,Number_Of_Phi_values))


# #print (z_BPC_Reshaped) #definitely not. Because these should all be the same value. 

# #Check to see if the direction of the projected cone and expected direction are the same. 

# BPC_expected = [r*np.sin(theta_rot)*np.cos(phi_rot), r*np.sin(theta_rot)*np.sin(phi_rot), r*np.cos(theta_rot)]
# mag_BPC_expected = np.sqrt (BPC_expected[0]*BPC_expected[0] + BPC_expected[1]*BPC_expected[1] + BPC_expected[2]*BPC_expected[2])

# x_mean = []
# y_mean = []
# z_mean = []
# BPC_mean = []
# mag_BPC_mean = []
# mag_multiplied = []
# Dot_BPC = []
# theta_difference = []
# Direction_Rank = []
# Surface_Check = []

# Total_On_Sphere = [0,0,0,0] #create a 4 x 1 empty matrix

# Position_Rank = []
# Total_Rank = []


# surface_allowed_error = .1

# for i in range (0,4):
#     #The following is for the direction match. However, since the sets of the solutions only cover half the range, we will choose to ignore this rating.
#     #x_mean.append(np.mean(x_BPC_Reshaped[i]))
#     #y_mean.append(np.mean(y_BPC_Reshaped[i]))
#     #z_mean.append(np.mean(z_BPC_Reshaped[i]))
#     #BPC_mean.append([x_mean[i],y_mean[i],z_mean[i]])
#     #mag_BPC_mean.append(np.sqrt(x_mean[i]*x_mean[i] + y_mean[i]*y_mean[i] + z_mean[i]*z_mean[i]))
#     #mag_multiplied.append(mag_BPC_expected * mag_BPC_mean[i])
#     #Dot_BPC.append(np.dot(BPC_expected,BPC_mean[i]))
#     #theta_difference.append(np.arccos(Dot_BPC[i]/mag_multiplied[i])*180/np.pi)
#     #print (theta_difference)
#     #Direction_Rank.append(100*(theta_difference[i])) #The closer this value is to 1 the better the rating of the direction
#     for j in range (0,Number_Of_Phi_values):
#         Surface_Check.append(np.sqrt(x_BPC_Reshaped[i][j]*x_BPC_Reshaped[i][j] + y_BPC_Reshaped[i][j]*y_BPC_Reshaped[i][j] + z_BPC_Reshaped[i][j]*z_BPC_Reshaped[i][j]))


# Surface_Check_Reshaped = np.reshape(Surface_Check, (4,Number_Of_Phi_values))

# #print (Surface_Check_Reshaped)


# for i in range (0,4):
#     for j in range (0,Number_Of_Phi_values):
#         if np.abs((r - (Surface_Check_Reshaped[i][j]))) < surface_allowed_error:  #This is not working 
#             Total_On_Sphere[i] = 1 + Total_On_Sphere[i] 
#     Position_Rank.append(100*(Total_On_Sphere[i]/Number_Of_Phi_values)) #The smaller the better. 
#     #Total_Rank.append(np.sqrt(Position_Rank[i]*Position_Rank[i] + Direction_Rank[i]*Direction_Rank[i])) 

# print (Position_Rank)




# Correct_Indicies = np.argpartition(Position_Rank, -2)[-2:]

# if np.sum(Position_Rank)/100 > 2:
#     Correct_Indicies = [1,3]


# x_BPC_Final = np.concatenate((x_BPC_Reshaped[Correct_Indicies[0]],x_BPC_Reshaped[Correct_Indicies[1]]))
# y_BPC_Final = np.concatenate((y_BPC_Reshaped[Correct_Indicies[0]],y_BPC_Reshaped[Correct_Indicies[1]]))
# z_BPC_Final = np.concatenate((z_BPC_Reshaped[Correct_Indicies[0]],z_BPC_Reshaped[Correct_Indicies[1]]))


# #if all position ranks are equal we need something to choose two odd values. 


# print (Correct_Indicies)   
# # #print(phi_ring)
# # print (x0_rot, y0_rot, z0_rot) 
# # # print (final_phi_set)
# # print (final_p_set)
# # print(phi_ring)
# # print(x_set,y_set,z_set)
# # print (y_set)    
    
    
# #Correct_Index = Total_Rank.index(min(Total_Rank))        
# #print(danytest)
# #print (Surface_Check_Reshaped)
# #print (Total_On_Sphere)
# #print (Position_Rank) #It says they are all not on the sphere. 
# #print (Direction_Rank) # They are all in the thousands. 
# #print (Correct_Index)